In [1]:
import pandas as pd
import nltk
import random
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

### Resume

In [3]:
df_resumes = pd.read_csv('../PreProcessing/processed_data/Resume_proc_lemm.csv')
df_resumes.head()

,ID,Resume_str,Category
0,16852973,hr administrator marketing associate hr admini...,HR
1,22323967,hr specialist u hr operation summary versatile...,HR
2,33176873,hr director summary year experience recruiting...,HR
3,27018550,hr specialist summary dedicated driven dynamic...,HR
4,17812897,hr manager skill highlight hr skill hr departm...,HR


In [4]:
df_resumes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2483 entries, 0 to 2482
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          2483 non-null   int64 
 1   Resume_str  2483 non-null   object
 2   Category    2483 non-null   object
dtypes: int64(1), object(2)
memory usage: 58.3+ KB


### Job Description

In [6]:
df_jobs = pd.read_csv('../PreProcessing Jobs/processed_data/Job_proc_lemm.csv')
df_jobs.head()

,job_id,title,description,skills_desc
0,921716,Marketing Coordinator,job descriptiona leading real estate firm new ...,Requirements: \n\nWe are seeking a College or ...
1,1829192,Mental Health Therapist/Counselor,aspen therapy wellness committed serving clien...,NaN
2,10998357,Assitant Restaurant Manager,national exemplar accepting application assist...,We are currently accepting resumes for FOH - A...
3,23221523,Senior Elder Law / Trusts and Estates Associat...,senior associate attorney elder law trust esta...,This position requires a baseline understandin...
4,91700727,Economic Development and Planning Intern,job summary economic development planning inte...,NaN


In [7]:
df_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122545 entries, 0 to 122544
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   job_id       122545 non-null  int64 
 1   title        122545 non-null  object
 2   description  122545 non-null  object
 3   skills_desc  2409 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.7+ MB


### Similarity

In [8]:
# Conversione in un formato utile a BERT
resumes = df_resumes["Resume_str"].tolist()
    
cat = df_resumes["Category"].astype("category")
labels = cat.cat.codes
category_names = list(cat.cat.categories)

In [9]:
sbert_model = SentenceTransformer('all-MiniLM-L12-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\andre\anaconda3\envs\ResumeProject\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\andre\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
resumes_embed = sbert_model.encode(df_resumes['Resume_str'], show_progress_bar=True)

Batches:   0%|          | 0/78 [00:00<?, ?it/s]

### Test

In [ ]:
#job_sample = df_jobs.sample(n=1).iloc[0]
#job_desc = job_sample['description']

In [18]:
job_sample = df_jobs[df_jobs['job_id'] == 3901356495].iloc[0] 
job_desc = job_sample['description']

In [19]:
job_embed = sbert_model.encode([job_desc], show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
similarity_vector = cosine_similarity(job_embed, resumes_embed).flatten()

In [21]:
print(similarity_vector)

[0.5511511  0.4935133  0.39638728 ... 0.5754805  0.6279856  0.42528796]


In [22]:
top_n = 5

job_real_id = job_sample['job_id']
job_title   = job_sample['title']


top_matches = similarity_vector.argsort()[::-1][:top_n]

results = []
for rank, cv_idx in enumerate(top_matches, start=1):
    cv_real_id   = df_resumes.iloc[cv_idx]['ID']
    cv_category  = df_resumes.iloc[cv_idx]['Category']
    score        = similarity_vector[cv_idx]
    
    results.append({
        'job_id':           job_real_id,
        'job_title':        job_title,
        'cv_id':            cv_real_id,
        'cv_category':      cv_category,
        'similarity_score': score,
        'rank':             rank
    })

df_matches = pd.DataFrame(results)
df_matches


,job_id,job_title,cv_id,cv_category,similarity_score,rank
0,3901356495,LINUX SYSTEM ADMINISTRATOR (CAOC),44115326,ADVOCATE,0.732067,1
1,3901356495,LINUX SYSTEM ADMINISTRATOR (CAOC),17025292,CONSULTANT,0.729244,2
2,3901356495,LINUX SYSTEM ADMINISTRATOR (CAOC),63989974,AUTOMOBILE,0.712114,3
3,3901356495,LINUX SYSTEM ADMINISTRATOR (CAOC),35389360,ENGINEERING,0.708131,4
4,3901356495,LINUX SYSTEM ADMINISTRATOR (CAOC),22450718,INFORMATION-TECHNOLOGY,0.696777,5


In [23]:
job_sample['description']

'mission support compute architecture operation center caoc quantum research provides program management network operation security operation technical support necessary advance service capability infrastructure technology responsibility quantum research intl quantum seeking linux administrator support team administrator standard linux system administration duty caoc huntsville al selected candidate perform standard linux system administration duty required maintain smooth operation multi user computer system consisting linux based application license server virtual machine shared service coordination network storage administrator required additional duty setting administrator service account maintaining system documentation tuning system performance installing system wide software allocate mass storage space developing monitoring policy standard allocation related use computing resource participating installation integration acceptance testing going maintenance system software environ